In [1]:
import graphviz
from datetime import datetime
import json
import numpy as np
import time

In [2]:
names = ['smallRandom','xsmallComplex','smallComplex','mediumRandom','MediumComplex','largeComplex','xlargeComplex','xxlargeComplex']
def create_path(names):
    paths = []
    for name in names : 
        path = './graphs/{}.json'.format(name)
        paths.append(path)
    return paths
paths = create_path(names)
print(paths[0])

./graphs/smallRandom.json


In [3]:
print(paths)
file_num = 0

['./graphs/smallRandom.json', './graphs/xsmallComplex.json', './graphs/smallComplex.json', './graphs/mediumRandom.json', './graphs/MediumComplex.json', './graphs/largeComplex.json', './graphs/xlargeComplex.json', './graphs/xxlargeComplex.json']


In [4]:
with open(paths[file_num], 'r') as f:
    data = json.load(f)

nodes = data['nodes']

In [5]:
def create_successors():
    ready_to_start = {}
    for node in nodes:
        nodes[node]['Successors']=[]
    for node in nodes :
        depend = nodes[node]['Dependencies']
        if len(depend)>0:
            for i in depend :
                nodes[str(i)]['Successors'].append(node)
        else :
            ready_to_start[node]=0
            # print(node)
    return ready_to_start

In [6]:
# create_successors()
# nodes

In [7]:
def weight(node):
    pt = node['Data'].split(':')
    time_s = float(pt[0])*3600 + float(pt[1])*60 + float(pt[2])
    return time_s

In [8]:
def add_ranks():
    for node in nodes:
        nodes[node]['rank'] = weight(nodes[node])
        nodes[node]['rank_calculated'] = False
        nodes[node]['finished'] = False

# for node in nodes.values() :
#     print(node['rank'])
#     print(node['rank_calculated'])

In [9]:
def calculate_rank(node):
    if not nodes[node]['rank_calculated']:
        succ = nodes[node]['Successors']     
        rank = weight(nodes[node])
        if len(succ) > 0:
            for i in succ:
                if not nodes[str(i)]['rank_calculated']:
                    # print("couldn't calculate rank")
                    return 1  
            weights = [nodes[str(i)]['rank'] for i in succ]
            # print('node weight:',rank)
            # print('successors:',succ)
            # print('succ weights:',weights)
            max_weights = max(weights)
            # print('max:',max_weights)
            rank += max_weights
        nodes[node]['rank'] = rank
        # print('calculated rank:', rank)
        nodes[node]['rank_calculated'] = True
    return 0

In [10]:
def graph_update():
    count = 0
    for node in reversed(nodes.keys()):
        # print('node:',node)
        count += calculate_rank(node)
    # print('not calculated ranks:',count)    
    return count

In [11]:
def calculate_ranks():
    epoch = 1
    count = 1
    while count > 0:
        # print('epoch:',epoch)
        count = graph_update()
        epoch+=1
    # print('rank calculation done')

In [12]:
def make_priority_list():
    node_list = []
    rank_list = []
    for node in nodes:
        node_list.append(node)
        rank_list.append(nodes[node]['rank'])

    # print(node_list)
    # print(rank_list)
    # indexes = np.flip(np.argsort(rank_list))
    # sorted_nodes = [node_list[i] for i in indexes]
    sorted_nodes = [x for _,x in sorted(zip(rank_list,node_list), reverse=True)]
    return sorted_nodes

In [13]:
def process_graph():
    ready_to_start = create_successors()
    add_ranks()
    calculate_ranks()
    priority_list = make_priority_list()
    return ready_to_start, priority_list
    

In [14]:
start_time = time.time()
ready_to_start, priority_list = process_graph()
duration = time.time() - start_time
# print(priority_list)
# print(len(nodes))
# print(len(priority_list))
print('duration:', duration)


duration: 0.0009970664978027344


In [15]:
print(ready_to_start)
print(priority_list)

{'1': 0}
['1', '2', '5', '4', '7', '8', '9', '6', '10', '3']


In [16]:
# def calculate_execution_time(priority_list,n_cores):
#     execution_times = [0]*n_cores
#     for node in priority_list :
#         task_time = weight(nodes[node])
#         i = np.argmin(execution_times)
#         execution_times[i] += task_time
#     return execution_times

In [17]:
# core_times = calculate_execution_time(priority_list,2)
# print(core_times)
# print(min(core_times))


In [18]:
# times = []
# priority_lists=[]
# for path in paths :
#     print('proccessing graph:', path)   
#     with open(path, 'r') as f:
#         data = json.load(f)
#     nodes = data['nodes']
#     start_time = time.time()
#     priority_list = process_graph()
#     end_time = time.time()
#     print('duration :', end_time - start_time)
#     priority_lists.append(priority_list)
#     core_times = calculate_execution_time(priority_list,2)
#     execution_time = min(core_times)
#     print(execution_time)
#     times.append(execution_time)
#     del(nodes)

# print(times)

    

In [19]:
print(priority_list)
priority_list_save = priority_list.copy()
print(priority_list_save)

['1', '2', '5', '4', '7', '8', '9', '6', '10', '3']
['1', '2', '5', '4', '7', '8', '9', '6', '10', '3']


In [20]:
# import matplotlib.pyplot as plt
# plt.figure()
# plt.title('evolution of graph execution time')
# plt.plot(times)


In [21]:
def make_empty_schedule(n_cores):
    schedule = {i:[] for i in range(n_cores)}
    return schedule

n_cores = 2
schedule = make_empty_schedule(n_cores)
print(schedule)

{0: [], 1: []}


In [22]:


def get_end_times(schedule):
    end_times = {}
    cores_that_did_smth = []
    for core in schedule : 
        if len(schedule[core])==0 :
            end_times[core] = 0
        else :
            end_times[core] = schedule[core][-1]['end']
            cores_that_did_smth.append(core)

    return end_times, cores_that_did_smth

def update_ready_to_start(finished_task, task_end_time):
    # ready_to_start_copy = ready_to_start.copy()
    nodes[finished_task]['finished'] = True
    succ = nodes[finished_task]['Successors']
    if len(succ)>0:
        for node in succ:
            dep_done = [nodes[str(dep)]['finished'] for dep in nodes[node]['Dependencies']]
            if not (False in dep_done) :
                ready_to_start[node] = task_end_time

def gef_first_core_ready(end_times, cores_that_did_smth):
    end_times_copy = end_times.copy()
    print(end_times_copy)
    ind = min(end_times_copy, key=end_times.get)
    task_end_time = end_times_copy.pop(ind)
    if len(cores_that_did_smth)>0 :
        if ind in cores_that_did_smth : 
            finished_task = schedule[ind][-1]['task']
            update_ready_to_start(finished_task, task_end_time)
    else :
        print('no tasks done')
    return ind, task_end_time, end_times_copy

def get_next_task():
    task = priority_list[0]
    return task

def try_assign_process(task, core, start_time):
    if len(ready_to_start)>0 :
        if task in ready_to_start and ready_to_start[task]<=start_time :
            end_time = start_time + weight(nodes[task])
            schedule[core].append({'task':task, 'start':start_time, 'end':end_time})
            ready_to_start.pop(task)
            if len(priority_list)>0 :     
                for i in range(len(priority_list)):
                    if task == priority_list[i] : 
                        priority_list.pop(i)
                        return True
            return True    
    return False

def update_dead_time(dead_time_list, done, time_stamp, ind):
    dead_time_copy = dead_time_list.copy()
    if done and len(dead_time_copy)>0 :
        for dead_time in dead_time_copy :
            dead_time['end'] = time_stamp
    else :
        dead_time_copy.append({'core':ind, 'start':time_stamp, 'end':0})
    # for core in dead_time_copy :
    #     dead_time_copy[core]['duration'] = dead_time_copy[core]['end'] - dead_time_copy[core]['start']
    return dead_time_copy

def fill_schedule():
    dead_time_list = []
    end_times, cores_that_did_smth = get_end_times(schedule)
    print('end_times:', end_times)
    done = False
    while not done :
        ind, time_stamp, end_times = gef_first_core_ready(end_times, cores_that_did_smth)
        # print(ind, time_stamp)
        task = get_next_task()
        done = try_assign_process(task, ind, time_stamp)
        dead_time_list = update_dead_time(dead_time_list, done, time_stamp, ind)
    return dead_time_list

def find_dead_time(dead_time_list) :
    max_val = -1
    max_ind = -1
    for i in range(len(dead_time_list)):
        new_val = dead_time_list[i]['start']
        if new_val>max_val :
            max_val = new_val
            max_ind = i
    # idle_intervals = []
    # for interval in dead_time_copy :
    #     idle_intervals.append(interval['start'])
    # i = np.argmax(idle_intervals)
    return max_ind


def fill_dead_time(dead_time_list):
    dead_time_copy = dead_time_list.copy()
    
    while len(dead_time_copy) > 0 :
        i = find_dead_time(dead_time_copy)
        dead_time_end = dead_time_copy[i]['end']
        dead_time_start = dead_time_copy[i]['start']
        idle_core = dead_time_copy[i]['core']
        possible_tasks = {}
        print('tasks ready', ready_to_start)

        for task in ready_to_start :
            task_available_start = ready_to_start[task]
            start_time = max(dead_time_start,task_available_start)
            if task_available_start<=dead_time_end and weight(nodes[task])<= dead_time_end - start_time :
                possible_tasks[task]=nodes[task]['rank']
        
        if len(possible_tasks)>0 :
            print('possible tasks to fill dead time', possible_tasks)
            task_start_time = ready_to_start[task]
            task_to_do = max(possible_tasks, key=possible_tasks.get)
            print(task_to_do)
            start_time = max(dead_time_start,task_start_time)
            task_end_time = start_time + weight(nodes[task])
            try_assign_process(task_to_do, idle_core, start_time)
            if dead_time_start<task_start_time :
                dead_time_copy.append({'core':idle_core, 'start':dead_time_start, 'end':task_start_time})
            if task_end_time<dead_time_end :
                dead_time_copy.append({'core':idle_core, 'start':task_end_time, 'end':dead_time_end})
        dead_time_copy.pop(i) 
   

    


In [23]:
end_times, cores_that_did_smth = get_end_times(schedule)
print(end_times, cores_that_did_smth)


{0: 0, 1: 0} []


In [24]:
ind, task_end_time, end_times = gef_first_core_ready(end_times, cores_that_did_smth)
print(ind,task_end_time,end_times)

{0: 0, 1: 0}
no tasks done
0 0 {1: 0}


In [25]:
n_cores = 2
schedule = make_empty_schedule(n_cores)
priority_list = priority_list_save.copy()
all_done = False
while not all_done :
    all_done = True
    dead_time_list = fill_schedule()
    print(dead_time_list)
    fill_dead_time(dead_time_list)
    if len(ready_to_start)>0 :
        all_done = False
    else : 
        print('no task ready to start, verifying all tasks are done')  
        if len(priority_list)>0: 
            for node in nodes :
                if nodes[node]['finished']==False :
                    print('task not finished :', node)
                    all_done = False
    print(schedule)

    


end_times: {0: 0, 1: 0}
{0: 0, 1: 0}
no tasks done
[{'core': 0, 'start': 0, 'end': 0}]
tasks ready {}
no task ready to start, verifying all tasks are done
task not finished : 1
task not finished : 2
task not finished : 3
task not finished : 4
task not finished : 5
task not finished : 6
task not finished : 7
task not finished : 8
task not finished : 9
task not finished : 10
{0: [{'task': '1', 'start': 0, 'end': 2843.6552354}], 1: []}
end_times: {0: 2843.6552354, 1: 0}
{0: 2843.6552354, 1: 0}
{0: 2843.6552354}
[{'core': 1, 'start': 0, 'end': 2843.6552354}]
tasks ready {'3': 2843.6552354, '4': 2843.6552354}
{0: [{'task': '1', 'start': 0, 'end': 2843.6552354}, {'task': '2', 'start': 2843.6552354, 'end': 6499.7264863}], 1: []}
end_times: {0: 6499.7264863, 1: 0}
{0: 6499.7264863, 1: 0}
{0: 6499.7264863}
[{'core': 1, 'start': 0, 'end': 6499.7264863}]
tasks ready {'3': 2843.6552354, '4': 2843.6552354, '7': 6499.7264863}
possible tasks to fill dead time {'3': 2741.9822376}
3
tasks ready {'4': 2

In [26]:
def get_score_1(schedule):
    score = 0
    for core_list in schedule.values() :
        if len(core_list)>0 :
            new_score = core_list[-1]['end']
            if new_score>score :
                score = new_score
    return score

def get_score_2(schedule):
    score = 0
    for core_list in schedule.values() :
        for task in core_list :
            new_score = task['end']
            if new_score>score :
                score = new_score
    return score


In [27]:
get_score_2(schedule)


23580.439512200002

In [28]:
!pip install plotly==5.13.1

In [29]:
import pandas as pd

def trasnform_schedule_to_pd(schedule):
    table = []
    for core_id in schedule :
        for task in schedule[core_id] :
            rank_val = nodes[task['task']]['rank']
            val = dict(Task = task['task'], Core = core_id, Start = task['start'], End = task['end'], Rank = rank_val)
            table.append(val)
    return pd.DataFrame(table)        


In [30]:
print(nodes[task]['rank'])

NameError: name 'task' is not defined

In [ ]:
from pathlib import Path  
pd_schedule = trasnform_schedule_to_pd(schedule)
print(pd_schedule)
path_name = 'schedules/{}.csv'.format(names[file_num])
filepath = Path(path_name)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
pd_schedule.to_csv(filepath)  


  Task  Core         Start           End          Rank
0    1     0      0.000000   2843.655235  18045.347103
1    2     0   2843.655235   6499.726486  15201.691868
2    5     0   6499.726486  11565.517510  11545.620617
3    7     0  11565.517510  15444.220621   9131.282645
4    9     0  15444.220621  20696.800155   5252.579534
5   10     0  20696.800155  23580.439512   2883.639357
6    3     1   6499.726486   9241.708724   2741.982238
7    4     1   9241.708724  13407.933057   9282.232150
8    8     1  13407.933057  17004.123293   6479.829593
9    6     1  17004.123293  22120.131110   5116.007817
